### COM361 &mdash; Introdução a Otimização &mdash; 2024, Prof. Amit ###

# Chance Constrained Portfolio Optimization #

#### Wagner Franco da Silva Junior (wagnerjunior@poli.ufrj.br)

*****

### Índice

1. [Introdução](#1.-Introdução)
1. [Demonstração](#2.-Demonstração)
1. [Formulação](#3.-Formulação)
1. [Comparação com Markowitz](#4.-Comparação-com-Markowitz)
1. [Conclusão](#5.-Conclusão)

## 1. Introdução ##

Neste trabalho foi abordado e estudado o comportamento de uma carteira de investimentos usando uma restrição probabilística do problema para alocação ótima da carteira, onde o objetivo central foi estabeler um mínimo de retorno $R_{min}$ com um nível de confiança $\alpha$ para obtenção mais confiável dos dados. Essa formulação é muito útil em situações em que a segurança de um retorno mínimo é indispensável, como em investimentos institucionais ou estratégias conservadoras.

## 2. Demonstração ##

Considerando um LP na forma padrão e uma desigualdade linear incerta/probabilística, temos:

$$
\begin{aligned}
\underset{x}{\text{min}}\qquad& c^Tx \\
\text{sujeito a:}\qquad& a_i^T x \le b_i && i=1,\dots,m\\
\end{aligned}
$$

Onde $a_i$ não é conhecido mas sabemos que obedece a uma distribuição gaussiana com esperança $\bar{a}$ e matriz de covariância $\Sigma_i$ positiva definida. E a partir disso podemos provar que $a_i^T x$ também é uma variável aleatória com distribuição gaussiana e sua média e variância podem ser encontradas da seguinte forma:

### Cálculo da Esperança

$$
y = a_i^T x
$$

A esperança matemática de $ y $ é dada por:

$$
\mathbb{E}[y] = \mathbb{E}[a_i^T x]
$$

Como  $x$ é um vetor fixo e determinístico, ele pode ser retirado da expectativa. Temos:

$$
\mathbb{E}[a_i^T x] = x^T \mathbb{E}[a_i]
$$

Sabemos que $ a_i \sim \mathcal{N}(\bar{a}_i, \Sigma_i) $, portanto, a média de $ a_i $ é $ \bar{a}_i $. Substituindo:

$$
\mathbb{E}[a_i^T x] = x^T \bar{a}_i
$$

Por fim:

$$
\mathbb{E}[y] = \bar{a}_i^T x
$$

### Cálculo da Variância

$$
Var(y) = Var(a_i^T x)
$$

Como neste caso, x é um vetor fixo e $ a_i \sim \mathcal{N}(\bar{a}_i, \Sigma_i) $ pode-se aplicar a fórmula padrão para uma combinação linear de variáveis gaussianas que é:

$$
Var(a_i^T x) = x^T \Sigma_i x
$$


## 3. Formulação ##

Para formulação do problema, sabemos que a função objetivo do modelo é para maximizar o retorno esperado com a limitação da probabilidade de perda ao máximo $\alpha$ e de forma que as alocações de x satisfaça $\Sigma x_i$ e $x_i \ge 0$. Assim, basta transformar a restrição em SOCP e utilizar na forma padrão para otimização, da seguinte forma:

$$
Prob\{p^T x \geq R_{\text{min}}\} \geq \alpha
$$

onde:
- $ p $: Vetor de retornos médios.
- $ x $: Vetor de pesos de alocação nos ativos.
- $ R_{\text{min}} $: Retorno mínimo desejado.
- $ \alpha $: Nível de confiança ($ 0 < \alpha < 1 $).

---

A probabilidade acumulada é expressa como:

$$
Prob\left\{p^T x - R_{\text{min}} \geq \Phi^{-1}(\alpha) \cdot \sqrt{x^T \Sigma x}\right\} \geq 0
$$

$ \Phi^{-1}(\alpha) $ é a quantidade da distribuição normal padrão associada ao nível de confiança $ \alpha $.

---

A inequação acima pode ser reescrita em termos determinísticos como:

$$
p^T x - R_{\text{min}} \geq \Phi^{-1}(\alpha) \cdot \sqrt{x^T \Sigma x}
$$

Isso pode ser reorganizado para:

$$
p^T x - \Phi^{-1}(\alpha) \cdot \sqrt{x^T \Sigma x} \geq R_{\text{min}}
$$

---
Para que o problema possa ser resolvido usando otimização cônica, introduzimos uma variável auxiliar $ t $, tal que:

$$
t \geq \sqrt{x^T \Sigma x}
$$

A restrição $ p^T x - \Phi^{-1}(\alpha) \cdot t \geq R_{\text{min}} $ é linear em $ t $, e a condição $ t \geq \sqrt{x^T \Sigma x} $ pode ser expressa como uma restrição de cone de segunda ordem:

$$
\| \Sigma^{1/2} x \|_2 \leq t
$$

onde $ \| \cdot \|_2 $ denota a norma $ \ell_2 $.

---
Assim, o SOCP final fica da seguinte forma:

$$
\begin{aligned}
\underset{x}{\text{max}}\qquad& \bar{p}^T x \\
\text{sujeito a:}\qquad& p^T x - t \cdot \| \Sigma^{1/2} x \|_2 \geq R_{\text{min}},\\ \qquad& \Sigma x_i = 1, x_i \geq 0
\end{aligned}
$$

In [1]:
# Nesse script foi feito o modelo usando médias e variância aleatórias para testar, e foi obtido um retorno esperado de 11.9%
using JuMP
using COSMO
using LinearAlgebra
using Random
using Distributions

n = 5
Random.seed!(123)

p = rand(0.05:0.01:0.15, n)

Σ = Symmetric(rand(0.01:0.01:0.05, n, n))
Σ += Diagonal(ones(n)) * 0.1 

α = 0.95
Φ_inv = quantile(Normal(0, 1), α)
R_min = 0.08

model = Model(COSMO.Optimizer)

@variable(model, x[1:n] >= 0)
@variable(model, t >= 0)

@objective(model, Max, dot(p, x))

@constraint(model, sum(x) == 1)

@constraint(model, p'x - Φ_inv * t >= R_min)
@constraint(model, [Σ^(1/2) * x; t] in SecondOrderCone())

optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("Solução ótima encontrada:")
    println("Alocação: ", value.(x))
    println("Retorno esperado: ", dot(p, value.(x)))
else
    println("Falha na obtenção de solução ótima.")
end

## 4. Comparação com Markowitz ##

A comparação com o QP de Markowitz pode ser feita de diversas formas, o seguinte script retorna um valor esperado de 0.327% com um risco de 4.46% que são ligeiramente melhores que o de Markowitz, principalmente ao considerarmos que para o valor esperado chegar perto disso (0.34% no Markowitz) foi necessário colocar um risco relativamente algo no valor de $\lambda$, de forma que torna inviável correr tanto riscos para ter retorno, que mostra um primeiro ponto a favor do modelo probabilístico, apesar dos riscos grandes do mesmo, mas com um valor de confiabilidade alto isso pode ser resolvido. Além do probabilístico ser mais robusto a incertezas justamente por ser uma parte fundamental do próprio modelo.

In [ ]:
using JuMP
using COSMO
using LinearAlgebra
using CSV
using DataFrames
using Distributions
using PyPlot

p_df = CSV.read("folio_mean.csv", DataFrame)
p = Matrix(p_df)[:, 1]

Σ_df = CSV.read("folio_cov.csv", DataFrame)
Σ = Matrix(Σ_df[:, 2:end])
Σ = Symmetric(Σ)

n = length(p)

α = 0.95
Φ_inv = quantile(Normal(0, 1), α)
R_min = 0.0031

model = Model(COSMO.Optimizer)

@variable(model, x[1:n] >= 0)
@variable(model, t >= 0)

@objective(model, Max, dot(p, x))
@constraint(model, sum(x) == 1)
@constraint(model, p'x - Φ_inv * t >= R_min)
@constraint(model, [Σ^(1/2) * x; t] in SecondOrderCone())

optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    xsol = value.(x)
    ret = dot(p, xsol)
    std = sqrt(dot(xsol, Σ * xsol))

    figure(figsize=(12, 6))
    bar(1:n, xsol, color="blue")
    title(string("Otimização - Portfolio\n",
                 "Retorno esperado (p): ", round(ret * 100, digits=3), "%, ",
                 "Risco (σ): ", round(std * 100, digits=2), "%"))
    xlabel("Ativos")
    ylabel("Alocação")
    xlim(0, n + 1)
    tight_layout()
    show()
else
    println("Falha na obtenção de solução ótima.")
end


## 5. Conclusão ##

Dessa forma, podemos concluir que o melhor modelo depende da situação em que vai ser aplicado, se precisar de um modelo mais robusto mas que sofra alguns riscos, o probabilístico se encaixa, caso queira um modelo mais simples que garanta que vai ter retorno dependendo do valor de risco que for aplicado, a escolha é o de Markowitz. A abordagem probabilística, ao incorporar distribuições acumuladas oferece maior proteção a cenários adversos como em mercados voláteis. Os resultados obtidos evidenciam que a probabilística, apesar de mais complexa computacionalmente, proporciona alocações de ativos mais confiáveis, especialmente em contextos onde incertezas e choques de mercado desempenham um papel relevante. Isso reforça sua aplicabilidade em cenários reais, onde retornos frequentemente não seguem distribuições normais e eventos extremos podem impactar significativamente o desempenho dos portfólios